# TextData 
## Data Modeling

### 预处理部分

In [15]:
import html
from html.parser import HTMLParser
import string
import re
import itertools
from autocorrect import Speller
import nltk
from nltk.corpus import stopwords
import jieba
import re
from textblob import TextBlob


In [5]:
#读入文件
import csv
import itertools
def _read_file(input_file, quotechar=None):
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t', quotechar=quotechar)
        lines = []
        for line in reader:
            if sys.version_info[0] == 2:
                line = list(unicode(cell, 'utf-8') for cell in line)
            lines.append(line)
        content=''.join(list(itertools.chain(*lines)))
    return content

In [104]:
a=_read_file('Chinese_text.txt')
a

'乳腺癌是乳腺上皮细胞在多种致癌因子的作用下，发生增殖失控的现象。疾病早期常表现为乳房肿块、乳头溢液、腋窝淋巴结肿大等症状，晚期可因癌细胞发生远处转移，出现多器官病变，直接威胁患者的生命。乳腺癌常被称为“粉红杀手”，其发病率位居女性恶性肿瘤的首位，男性乳腺癌较为少见。随着医疗水平的提高，乳腺癌已成为疗效最佳的实体肿瘤之一。宫颈癌和乳腺癌并称女性两大“隐性杀手”。结核病是由结核分枝杆菌引起的慢性传染病，可侵及许多脏器，以肺部结核感染最为常见。排菌者为其重要的传染源。人体感染结核菌后不一定发病，当抵抗力降低或细胞介导的变态反应增高时，才可能引起临床发病。若能及时诊断，并予合理治疗，大多可获临床痊愈。'

In [47]:
b=_read_file('text2.txt')
b

"I enjoyd the event which took place yesteday & I lovdddd itttt ! The link to the show is http://t.co/4ftYom0i It's awesome you'll luv it #HadFun #Enjoyed BFN GNBest of all, NLTK is a free, open source, community-driven project.NLTK has been called “a wonderful tool for teaching, and working in, computational linguistics using Python,” and “an amazing library to play with natural language.”"

In [48]:
e=_read_file('text3.txt')
e

"Python is a high-level, interpreted, interactive and object-oriented scripting language. Python is designed to be highly readable. It uses English keywords frequently where as other languages use punctuation, and it has fewer syntactical constructions than other languages.Python was developed by Guido van Rossum in the late eighties and early nineties at the National Research Institute for Mathematics and Computer Science in the Netherlands.Python is derived from many other languages, including ABC, Modula-3, C, C++, Algol-68, SmallTalk, and Unix shell and other scripting languages.Python is copyrighted. Like Perl, Python source code is now available under the GNU General Public License (GPL).Python is now maintained by a core development team at the institute, although Guido van Rossum still holds a vital role in directing its progress.Another input is the mask image (cloud.png). The final Result is on the right side.The experience was bad as hell.This app is really helpful.Damn the 

In [9]:
#去掉网址，#等非英文字符的内容
def rm_nontext(text):
    text_rmurl=html.unescape(text)
    text = re.sub(r'https?:\/\/.\S+', "", text_rmurl)
    text = re.sub(r'#', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    return text
 #缩写还原,用完整全称替换;分开单词,比如forthewin;统一转换为小写；俚语转换；词形还原；拼写检查
def conv_text(text):
    Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
            "'d":" would","'ve":" have","'re":" are"}
    for key,value in Apos_dict.items():
        if key in text:
            text=text.replace(key,value)        
        
    text = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",text) if s])

    text=text.lower()

    file=open("./slang.txt","r")
    slang=file.read()
    slang=slang.split('\n')
    text_tokens=text.split()
    slang_word=[]
    meaning=[]

    for line in slang:
        temp=line.split("=")
        slang_word.append(temp[0])
        meaning.append(temp[-1])

    for i,word in enumerate(text_tokens):
        if word in slang_word:
            idx=slang_word.index(word)
            text_tokens[i]=meaning[idx]
            
    text=" ".join(text_tokens)

    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    spell = Speller(lang='en')
    #spell check
    text=spell(text)
    return text
#去停用词
def rm_stopwords(text):
    stopwords_eng = stopwords.words('english')
    
    text_tokens=text.split()
    text_list=[]
    #remove stopwords
    for word in text_tokens:
        if word not in stopwords_eng:
            text_list.append(word)
    return text_list
#去标点
def rm_punc(text_list):
    clean_text=[]
    #remove punctuations
    for word in text_list:
        if word not in string.punctuation:
            clean_text.append(word)
    return clean_text

In [10]:
tweet_rm1=rm_nontext(e)
tweet_rm2=conv_text(tweet_rm1)
# print(tweet_rm2)
textls=rm_stopwords(tweet_rm2)
# print(textls)
en_text=rm_punc(textls)
print(en_text)

['python', 'high-level,', 'interpreted,', 'interactive', 'object-oriented', 'scripting', 'language.', 'python', 'designed', 'highly', 'readable.', 'uses', 'english', 'keywords', 'frequently', 'languages', 'use', 'punctuation,', 'fewer', 'syntactic', 'constructions', 'languages.', 'python', 'developed', 'guide', 'van', 'possum', 'late', 'eighties', 'early', 'nineties', 'national', 'research', 'institute', 'mathematics', 'computer', 'science', 'netherlands.', 'python', 'derived', 'many', 'languages,', 'including', 'abc,', 'module-3,', 'c,', 'c++,', 'algal-68,', 'small', 'talk,', 'unix', 'shell', 'scripting', 'languages.', 'python', 'copyrighted.', 'like', 'perl,', 'python', 'source', 'code', 'available', 'gnu', 'general', 'public', 'license', 'gpl).', 'python', 'maintained', 'core', 'development', 'team', 'institute,', 'although', 'guide', 'van', 'possum', 'still', 'holds', 'vital', 'role', 'directing', 'progress.', 'another', 'input', 'mask', 'image', '(cloud.png).', 'final', 'result', 

In [12]:
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':  # 判断一个uchar是否是汉字
        return True
    else:
        return False
 
def allcontents(contents):
    content = ''
    for i in contents:
        if is_chinese(i):
            content = content+i
    # print('\n处理后的句子为:\n'+content)
#先去停用词，再分词
# 停用词表
def get_stopword():
    s=set()
    with open('StopwordsList.txt','r',encoding='UTF-8') as f:
        for line in f:
            s.add(line.strip())
    return s

# 去掉文本中的停用词
def drop_stopwords(words):
    content_dropped=[word for word in words if word not in stopword]
    content=''.join(list(itertools.chain(*content_dropped)))
    return content
# 对文本进行jieba分词
def fenci(datas):
    # cut_words = map(lambda s: list(jieba.cut(s)), datas)
    cut_words=jieba.cut(datas, cut_all=False, HMM=True)
    fenci_list=list(cut_words)
    print("总词汇量："+str(len(fenci_list)))
    return fenci_list

In [105]:

centents = '1,2,3...how ara you *_*开始吧， 加油！乳腺癌是乳腺上皮细胞在多种致癌因子的作用下，发生增殖失控的现象。疾病早期常表现为乳房肿块、乳头溢液、腋窝淋巴结肿大等症状，晚期可因癌细胞发生远处转移，出现多器官病变，直接威胁患者的生命。'
# print('原句子为:\n'+centents)
sentence = allcontents(centents)

stopword=get_stopword()
contents=drop_stopwords(a)
contents
fenci_con=fenci(contents)
print(fenci_con)

总词汇量：99
['乳腺癌', '乳腺', '上皮细胞', '致癌', '子作', '下', '发生', '增殖', '失控', '现象', '疾病', '早期', '常', '表现', '乳房', '肿块', '乳头', '溢液', '腋窝', '淋巴结', '肿大', '症状', '晚期', '癌细胞', '发生', '远处', '转移', '出现', '器官', '病变', '直接', '威胁', '患', '生命', '乳腺癌', '常称', '粉红', '杀手', '发病率', '位居', '女性', '恶性肿瘤', '首位', '男性', '乳腺癌', '少见', '医疗', '水平', '提高', '乳腺癌', '已成', '疗效', '最佳', '实体肿瘤', '宫颈癌', '乳腺癌', '称', '女性', '两大', '隐性', '杀手', '结核病', '结核', '分枝杆菌', '引', '慢性', '传染病', '侵许', '脏器', '肺部', '结核', '感染', '最', '常见', '排菌重', '传染源', '人体', '感染', '结核菌', '后', '不定', '发病', '抵抗力', '降低', '细胞', '介导', '变态反应', '增高', '时才', '引床', '发病', '时', '诊断', '予', '合理', '治疗', '大', '获床', '痊愈']


### sentiment analysis

In [49]:
#英文
text=e
# create a TextBlob object
blob_object = TextBlob(text)
text_sentence=blob_object.sentences

In [50]:
text_sentence

[Sentence("Python is a high-level, interpreted, interactive and object-oriented scripting language."),
 Sentence("Python is designed to be highly readable."),
 Sentence("It uses English keywords frequently where as other languages use punctuation, and it has fewer syntactical constructions than other languages.Python was developed by Guido van Rossum in the late eighties and early nineties at the National Research Institute for Mathematics and Computer Science in the Netherlands.Python is derived from many other languages, including ABC, Modula-3, C, C++, Algol-68, SmallTalk, and Unix shell and other scripting languages.Python is copyrighted."),
 Sentence("Like Perl, Python source code is now available under the GNU General Public License (GPL).Python is now maintained by a core development team at the institute, although Guido van Rossum still holds a vital role in directing its progress.Another input is the mask image (cloud.png)."),
 Sentence("The final Result is on the right side.T

In [58]:
def split_sentence(text_str):
    sen_list=[]
    nlp = spacy.load("en_core_web_sm") 
    doc = nlp(text_str)   
    for sent in doc.sents:
        s=str(sent)
        sen_list.append(s)
    return sen_list

split_text=split_sentence(e)
print(split_text)

['Python is a high-level, interpreted, interactive and object-oriented scripting language.', 'Python is designed to be highly readable.', 'It uses English keywords frequently where as other languages use punctuation, and it has fewer syntactical constructions than other languages.', 'Python was developed by Guido van Rossum in the late eighties and early nineties at the National Research Institute for Mathematics and Computer Science in the Netherlands.', 'Python is derived from many other languages, including ABC, Modula-3, C, C++, Algol-68, SmallTalk, and Unix shell and other scripting languages.', 'Python is copyrighted.', 'Like Perl, Python source code is now available under the GNU General Public License (GPL).Python is now maintained by a core development team at the institute, although Guido van Rossum still holds a vital role in directing its progress.', 'Another input is the mask image (cloud.png).', 'The final Result is on the right side.', 'The experience was bad as hell.', 

In [59]:
def sentiment_ana(sentence_list):
    feedbacks = sentence_list
    positive_feedbacks = []
    negative_feedbacks = []

    for feedback in feedbacks:
        feedback_polarity = TextBlob(feedback).sentiment.polarity
        if feedback_polarity > 0:
            positive_feedbacks.append(feedback)
            continue
        negative_feedbacks.append(feedback)

    print('Positive_feebacks Count : {}'.format(len(positive_feedbacks)))
    print(positive_feedbacks)
    print('Negative_feedback Count : {}'.format(len(negative_feedbacks)))
    print(negative_feedbacks)

In [60]:
sentiment_ana(split_text)

Positive_feebacks Count : 6
['Python is designed to be highly readable.', 'Python is derived from many other languages, including ABC, Modula-3, C, C++, Algol-68, SmallTalk, and Unix shell and other scripting languages.', 'Like Perl, Python source code is now available under the GNU General Public License (GPL).Python is now maintained by a core development team at the institute, although Guido van Rossum still holds a vital role in directing its progress.', 'The final Result is on the right side.', 'This app is really helpful.', 'I love the app is amazing .']
Negative_feedback Count : 8
['Python is a high-level, interpreted, interactive and object-oriented scripting language.', 'It uses English keywords frequently where as other languages use punctuation, and it has fewer syntactical constructions than other languages.', 'Python was developed by Guido van Rossum in the late eighties and early nineties at the National Research Institute for Mathematics and Computer Science in the Nethe

### 文本相似度

In [61]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [81]:
sentences = ["The bottle is empty",
"There is nothing in the bottle"]
sentences_jac = [sent.lower().split(" ") for sent in sentences]
jaccard_similarity(sentences_jac[0], sentences_jac[1])

0.42857142857142855

In [93]:
from math import sqrt, pow, exp
 
def squared_sum(x):
  """ return 3 rounded square rooted value """
  return round(sqrt(sum([a*a for a in x])),3)
 
def euclidean_distance(x,y):
  """ return euclidean distance between two lists """
  return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

def distance_to_similarity(distance):
  return 1/(exp(distance))

In [94]:
embeddings = [nlp(sentence).vector for sentence in sentences]
distance = euclidean_distance(embeddings[0], embeddings[1])
print(distance)#计算出距离后需要转换成相似度
eu_similarity=distance_to_similarity(distance) 
print('运用欧氏距离计算的相似度'+str(eu_similarity))
#欧式距离计算相似度再检查一下

1.7483274802858235
运用欧氏距离计算的相似度0.17406482698308973


### LDA

In [107]:
# fenci_con

In [125]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import codecs
def lda_model(fenci_con):
    train = []
    for w in fenci_con:
        train.append([w])
    dictionary = corpora.Dictionary(train)
    corpus = [dictionary.doc2bow(text) for text in train]
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=60)
    # num_topics：主题数目
    # passes：训练伦次
    # num_words：每个主题下输出的term的数目

    for topic in lda.print_topics(num_words = 10):
        termNumber = topic[0]
        print(topic[0], ':', sep='')
        listOfTerms = topic[1].split('+')
        for term in listOfTerms:
            listItems = term.split('*')
            print('  ', listItems[1], '(', listItems[0], ')', sep='')

    print('\nPerplexity: ', lda.log_perplexity(corpus))#The LDA model (lda_model) we have created above can be used to compute the model’s perplexity, i.e. how good the model is. The lower the score the better the model will be.
    import pyLDAvis.gensim_models
    d=pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)   
    pyLDAvis.save_html(d, 'lda_result.html')

In [126]:
lda_model(en_text)

0:
  "python" (0.092)
  "possum" ( 0.031)
  "guide" ( 0.031)
  "language." ( 0.018)
  "keywords" ( 0.018)
  "mathematics" ( 0.018)
  "c," ( 0.018)
  "input" ( 0.018)
  "punctuation," ( 0.018)
  "late"( 0.018)
1:
  "languages." (0.027)
  "scripting" ( 0.027)
  "van" ( 0.027)
  "still" ( 0.016)
  "interpreted," ( 0.016)
  "abc," ( 0.016)
  "including" ( 0.016)
  "netherlands." ( 0.016)
  "vital" ( 0.016)
  "module-3,"( 0.016)

Perplexity:  -5.303836750801632


In [127]:
lda_model(fenci_con)

0:
  "乳腺癌" (0.056)
  "感染" ( 0.025)
  "结核" ( 0.025)
  "威胁" ( 0.015)
  "引床" ( 0.015)
  "分枝杆菌" ( 0.015)
  "已成" ( 0.015)
  "介导" ( 0.015)
  "抵抗力" ( 0.015)
  "引"( 0.015)
1:
  "女性" (0.027)
  "杀手" ( 0.027)
  "发生" ( 0.027)
  "发病" ( 0.027)
  "男性" ( 0.016)
  "称" ( 0.016)
  "肿块" ( 0.016)
  "增殖" ( 0.016)
  "子作" ( 0.016)
  "降低"( 0.016)

Perplexity:  -5.441214442091983
